In [4]:
cd C:/Users/Tanner/Documents/repos/ironmarch-archetypal/

C:\Users\Tanner\Documents\repos\ironmarch-archetypal


In [5]:
import torch
import pandas as pd
import numpy as np
import os
from ast import literal_eval
import scipy

In [8]:
latent_dim = 16
n_neighbors = 10
embedding_path = 'saved_embeddings\\train_sinkhorn_vae_2021-08-12_19-45-38\\train_sinkhorn_vae_c774e2cc_6_one=0.94555,zero=0.8_2021-08-12_19-45-38\checkpoint_000040\embeddings.csv'

In [9]:
embeddings = pd.read_csv(embedding_path).to_dict(orient = 'records')

def convert_string_to_tensor(string):
    return np.array(string.strip('[').strip(']').split()).astype('float')

embeddings = [{'id' : item['id'], 'embedding' : convert_string_to_tensor(item['embedding'])} for item in embeddings]

embedding_array = []
id_list = []
for embedding in embeddings:
    embedding_array.append(embedding['embedding'])
    id_list.append(embedding['id'])
embedding_array = np.array(embedding_array)
print(embedding_array.shape)

(217757, 16)


In [10]:
archetypes = np.diag(np.array([1.0 for i in range(latent_dim)]))
distances = s

(217773, 16)


In [11]:
distance_matrix = distance.squareform(distance.pdist(all_points))
archetype_distance = distance_matrix[-16:]
archetype_closest = np.argsort(archetype_distance, axis = 1)
print(archetype_closest)

MemoryError: Unable to allocate 177. GiB for an array with shape (23712430878,) and data type float64

In [9]:
raw_posts = pd.read_csv('iron_march_201911\core_message_posts.csv')

In [10]:
output_directory = 'post_training_analysis/closest_posts/output'
output_file_name = embedding_path.split('\\')[-1].replace('.csv', '') + '.txt'
output_path = os.path.join(output_directory, output_file_name)

with open(output_path, 'w+', encoding = 'utf-8') as output_file:
    for num, archetype in enumerate(archetype_closest):
        output_file.write(f'---- ARCHETYPE {num} ----\n\n')
        for index in archetype[1: 1 + n_neighbors]:
            id = id_list[index]
            mask = raw_posts['msg_id'].values == id
            post = raw_posts.loc[mask]['msg_post'].values[0]
            output_file.write(f'MESSAGE {id}:\n{post.strip()}\n\n')